In [1]:
import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import WebKB
from torch_geometric.loader import DataLoader

In [2]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
#data.adj_t = data.adj_t.to_symmetric()
#data.adj_t = data.adj_t.to_symmetric()
print(data)
#split_idx = dataset.get_idx_split()
#train_idx = split_idx['train'].to(device)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [3]:
train_index = np.where(data.train_mask)[0]
print(len(train_index))
valid_index = np.where(data.val_mask)[0]
print(len(valid_index))
test_index = np.where(data.test_mask)[0]
print(len(test_index))

1200
800
510


# GCN using only domain Feature

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [5]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            #x=F.softmax(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [6]:
idx=[data.train_mask[i][0] for i in range(183)]
train_index = np.where(idx)[0]
print(train_index)

[  4   5   8  11  13  15  17  21  24  25  26  27  28  32  33  34  36  38
  40  41  46  47  48  49  50  52  55  62  63  65  67  68  69  71  73  77
  78  80  82  84  86  87  88  89  97 100 102 104 107 109 110 111 113 115
 117 118 119 121 122 124 125 127 128 131 133 135 136 137 141 142 145 146
 148 149 153 156 157 158 159 160 162 163 166 167 169 173 176 178 181]


In [7]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.0, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels, dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.3904, Train: 24.17%, Valid: 16.25% Test: 5.88%
Run: 01, Epoch: 02, Loss: 1.8516, Train: 30.83%, Valid: 15.00% Test: 11.76%
Run: 01, Epoch: 03, Loss: 1.5139, Train: 55.83%, Valid: 43.75% Test: 31.37%
Run: 01, Epoch: 04, Loss: 1.2444, Train: 63.33%, Valid: 51.25% Test: 45.10%
Run: 01, Epoch: 05, Loss: 1.1097, Train: 72.50%, Valid: 55.00% Test: 47.06%
Run: 01, Epoch: 06, Loss: 1.0159, Train: 71.67%, Valid: 62.50% Test: 49.02%
Run: 01, Epoch: 07, Loss: 0.9255, Train: 71.67%, Valid: 62.50% Test: 50.98%
Run: 01, Epoch: 08, Loss: 0.8810, Train: 71.67%, Valid: 61.25% Test: 54.90%
Run: 01, Epoch: 09, Loss: 0.8432, Train: 72.50%, Valid: 58.75% Test: 54.90%
Run: 01, Epoch: 10, Loss: 0.7929, Train: 75.00%, Valid: 56.25% Test: 50.98%
Run: 01, Epoch: 11, Loss: 0.7609, Train: 77.50%, Valid: 61.25% Test: 52.94%
Run: 01, Epoch: 12, Loss: 0.7220, Train: 76.67%, Valid: 58.75% Test: 50.98%
Run: 01, Epoch: 13, Loss: 0.6858, Train: 76.67%, Valid: 57.50% Test: 49.02%
Run: 01, Epoc

Run: 02, Epoch: 67, Loss: 0.0386, Train: 99.17%, Valid: 47.50% Test: 54.90%
Run: 02, Epoch: 68, Loss: 0.0384, Train: 99.17%, Valid: 47.50% Test: 54.90%
Run: 02, Epoch: 69, Loss: 0.0392, Train: 98.33%, Valid: 46.25% Test: 54.90%
Run: 02, Epoch: 70, Loss: 0.0392, Train: 99.17%, Valid: 45.00% Test: 54.90%
Run: 02, Epoch: 71, Loss: 0.0371, Train: 99.17%, Valid: 45.00% Test: 54.90%
Run: 02, Epoch: 72, Loss: 0.0343, Train: 99.17%, Valid: 45.00% Test: 54.90%
Run: 02, Epoch: 73, Loss: 0.0338, Train: 99.17%, Valid: 45.00% Test: 54.90%
Run: 02, Epoch: 74, Loss: 0.0344, Train: 99.17%, Valid: 45.00% Test: 54.90%
Run: 02, Epoch: 75, Loss: 0.0341, Train: 99.17%, Valid: 43.75% Test: 54.90%
Run: 02, Epoch: 76, Loss: 0.0331, Train: 99.17%, Valid: 43.75% Test: 54.90%
Run: 02, Epoch: 77, Loss: 0.0315, Train: 99.17%, Valid: 43.75% Test: 54.90%
Run: 02, Epoch: 78, Loss: 0.0302, Train: 99.17%, Valid: 42.50% Test: 54.90%
Run: 02, Epoch: 79, Loss: 0.0299, Train: 99.17%, Valid: 42.50% Test: 54.90%
Run: 02, Epo

Run: 04, Epoch: 33, Loss: 0.1452, Train: 95.83%, Valid: 55.00% Test: 49.02%
Run: 04, Epoch: 34, Loss: 0.1375, Train: 95.83%, Valid: 52.50% Test: 47.06%
Run: 04, Epoch: 35, Loss: 0.1294, Train: 95.83%, Valid: 52.50% Test: 47.06%
Run: 04, Epoch: 36, Loss: 0.1215, Train: 95.83%, Valid: 53.75% Test: 45.10%
Run: 04, Epoch: 37, Loss: 0.1140, Train: 96.67%, Valid: 53.75% Test: 43.14%
Run: 04, Epoch: 38, Loss: 0.1074, Train: 97.50%, Valid: 52.50% Test: 43.14%
Run: 04, Epoch: 39, Loss: 0.1018, Train: 97.50%, Valid: 51.25% Test: 43.14%
Run: 04, Epoch: 40, Loss: 0.0972, Train: 97.50%, Valid: 51.25% Test: 45.10%
Run: 04, Epoch: 41, Loss: 0.0931, Train: 97.50%, Valid: 51.25% Test: 45.10%
Run: 04, Epoch: 42, Loss: 0.0889, Train: 97.50%, Valid: 48.75% Test: 45.10%
Run: 04, Epoch: 43, Loss: 0.0850, Train: 97.50%, Valid: 48.75% Test: 45.10%
Run: 04, Epoch: 44, Loss: 0.0818, Train: 97.50%, Valid: 48.75% Test: 45.10%
Run: 04, Epoch: 45, Loss: 0.0793, Train: 97.50%, Valid: 48.75% Test: 45.10%
Run: 04, Epo

Run: 06, Epoch: 02, Loss: 1.0297, Train: 60.83%, Valid: 46.25% Test: 41.18%
Run: 06, Epoch: 03, Loss: 0.8672, Train: 60.83%, Valid: 45.00% Test: 41.18%
Run: 06, Epoch: 04, Loss: 0.7891, Train: 63.33%, Valid: 46.25% Test: 41.18%
Run: 06, Epoch: 05, Loss: 0.7283, Train: 62.50%, Valid: 43.75% Test: 43.14%
Run: 06, Epoch: 06, Loss: 0.6777, Train: 65.00%, Valid: 42.50% Test: 45.10%
Run: 06, Epoch: 07, Loss: 0.6309, Train: 69.17%, Valid: 45.00% Test: 45.10%
Run: 06, Epoch: 08, Loss: 0.5873, Train: 70.83%, Valid: 46.25% Test: 45.10%
Run: 06, Epoch: 09, Loss: 0.5467, Train: 75.83%, Valid: 47.50% Test: 41.18%
Run: 06, Epoch: 10, Loss: 0.5094, Train: 80.83%, Valid: 46.25% Test: 45.10%
Run: 06, Epoch: 11, Loss: 0.4749, Train: 85.00%, Valid: 46.25% Test: 47.06%
Run: 06, Epoch: 12, Loss: 0.4428, Train: 85.00%, Valid: 45.00% Test: 47.06%
Run: 06, Epoch: 13, Loss: 0.4140, Train: 85.83%, Valid: 47.50% Test: 47.06%
Run: 06, Epoch: 14, Loss: 0.3878, Train: 86.67%, Valid: 48.75% Test: 49.02%
Run: 06, Epo

Run: 07, Epoch: 70, Loss: 0.2157, Train: 89.17%, Valid: 47.50% Test: 41.18%
Run: 07, Epoch: 71, Loss: 0.2153, Train: 90.00%, Valid: 48.75% Test: 41.18%
Run: 07, Epoch: 72, Loss: 0.2227, Train: 89.17%, Valid: 48.75% Test: 39.22%
Run: 07, Epoch: 73, Loss: 0.2176, Train: 89.17%, Valid: 47.50% Test: 39.22%
Run: 07, Epoch: 74, Loss: 0.2223, Train: 89.17%, Valid: 47.50% Test: 39.22%
Run: 07, Epoch: 75, Loss: 0.2150, Train: 89.17%, Valid: 48.75% Test: 41.18%
Run: 07, Epoch: 76, Loss: 0.2096, Train: 89.17%, Valid: 48.75% Test: 41.18%
Run: 07, Epoch: 77, Loss: 0.2048, Train: 90.00%, Valid: 48.75% Test: 41.18%
Run: 07, Epoch: 78, Loss: 0.2019, Train: 89.17%, Valid: 48.75% Test: 41.18%
Run: 07, Epoch: 79, Loss: 0.2012, Train: 89.17%, Valid: 48.75% Test: 43.14%
Run: 07, Epoch: 80, Loss: 0.2019, Train: 89.17%, Valid: 47.50% Test: 45.10%
Run: 07, Epoch: 81, Loss: 0.2015, Train: 89.17%, Valid: 50.00% Test: 45.10%
Run: 07, Epoch: 82, Loss: 0.1999, Train: 89.17%, Valid: 50.00% Test: 45.10%
Run: 07, Epo

Run: 09, Epoch: 39, Loss: 0.2208, Train: 94.17%, Valid: 41.25% Test: 43.14%
Run: 09, Epoch: 40, Loss: 0.2093, Train: 95.00%, Valid: 42.50% Test: 41.18%
Run: 09, Epoch: 41, Loss: 0.1979, Train: 94.17%, Valid: 42.50% Test: 41.18%
Run: 09, Epoch: 42, Loss: 0.1895, Train: 95.00%, Valid: 41.25% Test: 37.25%
Run: 09, Epoch: 43, Loss: 0.1805, Train: 95.00%, Valid: 41.25% Test: 39.22%
Run: 09, Epoch: 44, Loss: 0.1711, Train: 95.83%, Valid: 42.50% Test: 41.18%
Run: 09, Epoch: 45, Loss: 0.1616, Train: 95.83%, Valid: 42.50% Test: 41.18%
Run: 09, Epoch: 46, Loss: 0.1548, Train: 95.83%, Valid: 42.50% Test: 41.18%
Run: 09, Epoch: 47, Loss: 0.1466, Train: 95.83%, Valid: 43.75% Test: 43.14%
Run: 09, Epoch: 48, Loss: 0.1393, Train: 96.67%, Valid: 42.50% Test: 43.14%
Run: 09, Epoch: 49, Loss: 0.1317, Train: 96.67%, Valid: 43.75% Test: 43.14%
Run: 09, Epoch: 50, Loss: 0.1239, Train: 96.67%, Valid: 43.75% Test: 45.10%
Run: 09, Epoch: 51, Loss: 0.1174, Train: 97.50%, Valid: 43.75% Test: 45.10%
Run: 09, Epo

# Wise Embedding

In [8]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [9]:
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,1694,1695,1696,1697,1698,1699,1700,1701,1702,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [10]:
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.2)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [11]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    #print(f)
    f.clear()
    Fec.append(vec)

In [ ]:
print(Fec)

In [12]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [ ]:
print(SFec)

In [13]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[46., 72., 68.,  ..., 26., 42., 38.],
        [39., 47., 49.,  ..., 34., 39., 32.],
        [42., 48., 51.,  ..., 34., 39., 35.],
        ...,
        [22., 29., 34.,  ..., 18., 23., 22.],
        [53., 86., 77.,  ..., 24., 38., 29.],
        [50., 72., 86.,  ..., 21., 27., 26.]])


'torch.FloatTensor'

# W-GCN

In [14]:
data.x=CC_domain
print(data)

Data(x=[251, 10], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [15]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 100, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = WebKB(root='/tmp/Texas', name='Texas',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    #idx_train=[data.train_mask[i][0] for i in range(len(data.y))]
    #train_idx = np.where(idx_train)[0]
    #idx_val=[data.val_mask[i][0] for i in range(len(data.y))]
    #valid_idx = np.where(idx_val)[0]
    #idx_test=[data.test_mask[i][0] for i in range(len(data.y))]
    #test_idx = np.where(idx_test)[0]
    
    model = GAT(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.1641, Train: 4.17%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 02, Loss: 2.0564, Train: 4.17%, Valid: 6.25% Test: 1.96%
Run: 01, Epoch: 03, Loss: 1.8671, Train: 3.33%, Valid: 7.50% Test: 1.96%
Run: 01, Epoch: 04, Loss: 1.7917, Train: 3.33%, Valid: 7.50% Test: 1.96%
Run: 01, Epoch: 05, Loss: 1.6677, Train: 2.50%, Valid: 7.50% Test: 3.92%
Run: 01, Epoch: 06, Loss: 1.6619, Train: 3.33%, Valid: 6.25% Test: 7.84%
Run: 01, Epoch: 07, Loss: 1.6273, Train: 8.33%, Valid: 10.00% Test: 19.61%
Run: 01, Epoch: 08, Loss: 1.5613, Train: 14.17%, Valid: 11.25% Test: 17.65%
Run: 01, Epoch: 09, Loss: 1.5361, Train: 14.17%, Valid: 11.25% Test: 17.65%
Run: 01, Epoch: 10, Loss: 1.4813, Train: 15.00%, Valid: 11.25% Test: 17.65%
Run: 01, Epoch: 11, Loss: 1.4084, Train: 16.67%, Valid: 17.50% Test: 15.69%
Run: 01, Epoch: 12, Loss: 1.5021, Train: 20.00%, Valid: 20.00% Test: 21.57%
Run: 01, Epoch: 13, Loss: 1.4471, Train: 21.67%, Valid: 21.25% Test: 25.49%
Run: 01, Epoch: 14, Loss: 1.517

Run: 02, Epoch: 70, Loss: 1.0252, Train: 65.00%, Valid: 50.00% Test: 60.78%
Run: 02, Epoch: 71, Loss: 0.9969, Train: 65.00%, Valid: 51.25% Test: 62.75%
Run: 02, Epoch: 72, Loss: 1.1449, Train: 65.00%, Valid: 50.00% Test: 62.75%
Run: 02, Epoch: 73, Loss: 1.0968, Train: 62.50%, Valid: 51.25% Test: 64.71%
Run: 02, Epoch: 74, Loss: 1.0080, Train: 61.67%, Valid: 51.25% Test: 64.71%
Run: 02, Epoch: 75, Loss: 1.0747, Train: 58.33%, Valid: 46.25% Test: 64.71%
Run: 02, Epoch: 76, Loss: 1.0041, Train: 56.67%, Valid: 45.00% Test: 64.71%
Run: 02, Epoch: 77, Loss: 1.0806, Train: 58.33%, Valid: 46.25% Test: 64.71%
Run: 02, Epoch: 78, Loss: 1.0197, Train: 60.83%, Valid: 48.75% Test: 66.67%
Run: 02, Epoch: 79, Loss: 0.9663, Train: 60.83%, Valid: 48.75% Test: 66.67%
Run: 02, Epoch: 80, Loss: 1.0440, Train: 61.67%, Valid: 50.00% Test: 66.67%
Run: 02, Epoch: 81, Loss: 1.1113, Train: 62.50%, Valid: 50.00% Test: 66.67%
Run: 02, Epoch: 82, Loss: 1.0265, Train: 63.33%, Valid: 52.50% Test: 64.71%
Run: 02, Epo

Run: 04, Epoch: 54, Loss: 1.2708, Train: 48.33%, Valid: 50.00% Test: 35.29%
Run: 04, Epoch: 55, Loss: 1.2899, Train: 46.67%, Valid: 53.75% Test: 39.22%
Run: 04, Epoch: 56, Loss: 1.2857, Train: 45.83%, Valid: 60.00% Test: 41.18%
Run: 04, Epoch: 57, Loss: 1.2492, Train: 46.67%, Valid: 58.75% Test: 37.25%
Run: 04, Epoch: 58, Loss: 1.2505, Train: 49.17%, Valid: 55.00% Test: 37.25%
Run: 04, Epoch: 59, Loss: 1.2422, Train: 49.17%, Valid: 51.25% Test: 45.10%
Run: 04, Epoch: 60, Loss: 1.2126, Train: 50.83%, Valid: 40.00% Test: 47.06%
Run: 04, Epoch: 61, Loss: 1.1779, Train: 54.17%, Valid: 38.75% Test: 43.14%
Run: 04, Epoch: 62, Loss: 1.1859, Train: 53.33%, Valid: 33.75% Test: 37.25%
Run: 04, Epoch: 63, Loss: 1.1309, Train: 52.50%, Valid: 33.75% Test: 37.25%
Run: 04, Epoch: 64, Loss: 1.1465, Train: 51.67%, Valid: 38.75% Test: 39.22%
Run: 04, Epoch: 65, Loss: 1.1482, Train: 50.83%, Valid: 32.50% Test: 37.25%
Run: 04, Epoch: 66, Loss: 1.1824, Train: 52.50%, Valid: 32.50% Test: 39.22%
Run: 04, Epo

Run: 06, Epoch: 36, Loss: 1.3056, Train: 47.50%, Valid: 48.75% Test: 50.98%
Run: 06, Epoch: 37, Loss: 1.2532, Train: 47.50%, Valid: 47.50% Test: 49.02%
Run: 06, Epoch: 38, Loss: 1.2379, Train: 47.50%, Valid: 47.50% Test: 49.02%
Run: 06, Epoch: 39, Loss: 1.1944, Train: 46.67%, Valid: 46.25% Test: 50.98%
Run: 06, Epoch: 40, Loss: 1.2272, Train: 46.67%, Valid: 47.50% Test: 50.98%
Run: 06, Epoch: 41, Loss: 1.1795, Train: 47.50%, Valid: 47.50% Test: 50.98%
Run: 06, Epoch: 42, Loss: 1.2870, Train: 48.33%, Valid: 47.50% Test: 50.98%
Run: 06, Epoch: 43, Loss: 1.1811, Train: 51.67%, Valid: 45.00% Test: 47.06%
Run: 06, Epoch: 44, Loss: 1.2291, Train: 50.83%, Valid: 43.75% Test: 47.06%
Run: 06, Epoch: 45, Loss: 1.2275, Train: 51.67%, Valid: 43.75% Test: 45.10%
Run: 06, Epoch: 46, Loss: 1.1622, Train: 52.50%, Valid: 42.50% Test: 43.14%
Run: 06, Epoch: 47, Loss: 1.2316, Train: 51.67%, Valid: 43.75% Test: 43.14%
Run: 06, Epoch: 48, Loss: 1.2310, Train: 50.83%, Valid: 45.00% Test: 47.06%
Run: 06, Epo

Run: 08, Epoch: 20, Loss: 1.2901, Train: 47.50%, Valid: 47.50% Test: 50.98%
Run: 08, Epoch: 21, Loss: 1.2666, Train: 45.00%, Valid: 48.75% Test: 50.98%
Run: 08, Epoch: 22, Loss: 1.2169, Train: 45.00%, Valid: 50.00% Test: 52.94%
Run: 08, Epoch: 23, Loss: 1.2621, Train: 44.17%, Valid: 50.00% Test: 52.94%
Run: 08, Epoch: 24, Loss: 1.3007, Train: 46.67%, Valid: 52.50% Test: 54.90%
Run: 08, Epoch: 25, Loss: 1.3867, Train: 46.67%, Valid: 51.25% Test: 54.90%
Run: 08, Epoch: 26, Loss: 1.2289, Train: 47.50%, Valid: 52.50% Test: 54.90%
Run: 08, Epoch: 27, Loss: 1.2319, Train: 46.67%, Valid: 51.25% Test: 54.90%
Run: 08, Epoch: 28, Loss: 1.2197, Train: 49.17%, Valid: 50.00% Test: 54.90%
Run: 08, Epoch: 29, Loss: 1.3124, Train: 50.00%, Valid: 52.50% Test: 52.94%
Run: 08, Epoch: 30, Loss: 1.1891, Train: 50.00%, Valid: 51.25% Test: 49.02%
Run: 08, Epoch: 31, Loss: 1.3201, Train: 50.00%, Valid: 51.25% Test: 50.98%
Run: 08, Epoch: 32, Loss: 1.2957, Train: 47.50%, Valid: 51.25% Test: 50.98%
Run: 08, Epo

Run: 10, Epoch: 05, Loss: 1.4661, Train: 45.83%, Valid: 45.00% Test: 43.14%
Run: 10, Epoch: 06, Loss: 1.4836, Train: 45.83%, Valid: 47.50% Test: 47.06%
Run: 10, Epoch: 07, Loss: 1.4092, Train: 48.33%, Valid: 50.00% Test: 49.02%
Run: 10, Epoch: 08, Loss: 1.3080, Train: 53.33%, Valid: 45.00% Test: 52.94%
Run: 10, Epoch: 09, Loss: 1.3207, Train: 48.33%, Valid: 42.50% Test: 52.94%
Run: 10, Epoch: 10, Loss: 1.3019, Train: 51.67%, Valid: 40.00% Test: 52.94%
Run: 10, Epoch: 11, Loss: 1.3642, Train: 51.67%, Valid: 38.75% Test: 49.02%
Run: 10, Epoch: 12, Loss: 1.2711, Train: 51.67%, Valid: 37.50% Test: 50.98%
Run: 10, Epoch: 13, Loss: 1.2623, Train: 50.00%, Valid: 40.00% Test: 50.98%
Run: 10, Epoch: 14, Loss: 1.3974, Train: 50.83%, Valid: 38.75% Test: 50.98%
Run: 10, Epoch: 15, Loss: 1.2885, Train: 50.83%, Valid: 43.75% Test: 49.02%
Run: 10, Epoch: 16, Loss: 1.2628, Train: 49.17%, Valid: 43.75% Test: 47.06%
Run: 10, Epoch: 17, Loss: 1.3426, Train: 46.67%, Valid: 45.00% Test: 45.10%
Run: 10, Epo

# Topological Encodding 

In [4]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin')
data = dataset[0]
print(data)

Data(x=[251, 1703], edge_index=[2, 515], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10])


In [6]:

Number_nodes=len(data.y)
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [7]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [19]:
print(G.number_of_edges())

515


In [8]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [9]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [10]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

1  |  59 

2  |  53 

3  |  47 

4  |  27 

5  |  21 

6  |  17 

8  |  4 

9  |  5 

10  |  4 

11  |  5 

12  |  3 

13  |  2 

14  |  1 

18  |  1 

21  |  1 

122  |  1 



In [11]:
import pyflagser
Node_fil=[1,2,3,4,5,6,7,8,9,10,20,100]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=True, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 250 (100%)

In [24]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [15]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

# Topo-W-GCN

In [26]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[251, 10], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515], topo=[251, 24])


In [ ]:
print(len(topo_fe[0]))

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels, cached=True))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, cached=True))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels, cached=True))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x
        #return x.log_softmax(dim=-1)

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [29]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}
    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.01)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.01)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.6350, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 02, Loss: 1.4040, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 03, Loss: 1.4125, Train: 37.50%, Valid: 41.25% Test: 52.94%
Run: 01, Epoch: 04, Loss: 1.3070, Train: 31.67%, Valid: 25.00% Test: 23.53%
Run: 01, Epoch: 05, Loss: 1.3713, Train: 31.67%, Valid: 25.00% Test: 23.53%
Run: 01, Epoch: 06, Loss: 1.3289, Train: 51.67%, Valid: 50.00% Test: 54.90%
Run: 01, Epoch: 07, Loss: 1.3259, Train: 40.00%, Valid: 48.75% Test: 52.94%
Run: 01, Epoch: 08, Loss: 1.2617, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 09, Loss: 1.2509, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 10, Loss: 1.3029, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 11, Loss: 1.2718, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 12, Loss: 1.2819, Train: 41.67%, Valid: 51.25% Test: 52.94%
Run: 01, Epoch: 13, Loss: 1.2633, Train: 40.83%, Valid: 50.00% Test: 52.94%
Run: 01, Epo

Run: 01, Epoch: 125, Loss: 0.9212, Train: 65.83%, Valid: 57.50% Test: 50.98%
Run: 01, Epoch: 126, Loss: 0.8554, Train: 69.17%, Valid: 53.75% Test: 49.02%
Run: 01, Epoch: 127, Loss: 0.8884, Train: 69.17%, Valid: 52.50% Test: 47.06%
Run: 01, Epoch: 128, Loss: 0.8187, Train: 67.50%, Valid: 53.75% Test: 43.14%
Run: 01, Epoch: 129, Loss: 0.8610, Train: 61.67%, Valid: 58.75% Test: 41.18%
Run: 01, Epoch: 130, Loss: 0.8896, Train: 62.50%, Valid: 61.25% Test: 49.02%
Run: 01, Epoch: 131, Loss: 0.8859, Train: 61.67%, Valid: 62.50% Test: 50.98%
Run: 01, Epoch: 132, Loss: 0.8138, Train: 63.33%, Valid: 63.75% Test: 47.06%
Run: 01, Epoch: 133, Loss: 0.8089, Train: 64.17%, Valid: 61.25% Test: 45.10%
Run: 01, Epoch: 134, Loss: 0.8958, Train: 66.67%, Valid: 57.50% Test: 52.94%
Run: 01, Epoch: 135, Loss: 0.8512, Train: 65.83%, Valid: 57.50% Test: 50.98%
Run: 01, Epoch: 136, Loss: 0.8424, Train: 64.17%, Valid: 58.75% Test: 52.94%
Run: 01, Epoch: 137, Loss: 0.8569, Train: 65.00%, Valid: 62.50% Test: 50.98%

Run: 02, Epoch: 56, Loss: 0.7882, Train: 63.33%, Valid: 53.75% Test: 64.71%
Run: 02, Epoch: 57, Loss: 0.9100, Train: 64.17%, Valid: 52.50% Test: 64.71%
Run: 02, Epoch: 58, Loss: 0.8512, Train: 64.17%, Valid: 53.75% Test: 64.71%
Run: 02, Epoch: 59, Loss: 0.9511, Train: 65.83%, Valid: 53.75% Test: 62.75%
Run: 02, Epoch: 60, Loss: 0.8686, Train: 68.33%, Valid: 53.75% Test: 62.75%
Run: 02, Epoch: 61, Loss: 0.8515, Train: 68.33%, Valid: 53.75% Test: 62.75%
Run: 02, Epoch: 62, Loss: 0.8214, Train: 70.83%, Valid: 55.00% Test: 60.78%
Run: 02, Epoch: 63, Loss: 0.8629, Train: 70.00%, Valid: 55.00% Test: 58.82%
Run: 02, Epoch: 64, Loss: 0.9175, Train: 70.83%, Valid: 55.00% Test: 60.78%
Run: 02, Epoch: 65, Loss: 0.8245, Train: 70.83%, Valid: 53.75% Test: 58.82%
Run: 02, Epoch: 66, Loss: 0.8898, Train: 71.67%, Valid: 52.50% Test: 60.78%
Run: 02, Epoch: 67, Loss: 0.8148, Train: 70.00%, Valid: 52.50% Test: 60.78%
Run: 02, Epoch: 68, Loss: 0.8615, Train: 70.00%, Valid: 53.75% Test: 62.75%
Run: 02, Epo

Run: 02, Epoch: 187, Loss: 0.7024, Train: 69.17%, Valid: 55.00% Test: 60.78%
Run: 02, Epoch: 188, Loss: 0.6377, Train: 69.17%, Valid: 58.75% Test: 60.78%
Run: 02, Epoch: 189, Loss: 0.7777, Train: 68.33%, Valid: 56.25% Test: 60.78%
Run: 02, Epoch: 190, Loss: 0.7806, Train: 70.00%, Valid: 55.00% Test: 60.78%
Run: 02, Epoch: 191, Loss: 0.7020, Train: 68.33%, Valid: 56.25% Test: 58.82%
Run: 02, Epoch: 192, Loss: 0.6882, Train: 69.17%, Valid: 57.50% Test: 58.82%
Run: 02, Epoch: 193, Loss: 0.7261, Train: 69.17%, Valid: 55.00% Test: 60.78%
Run: 02, Epoch: 194, Loss: 0.6904, Train: 65.00%, Valid: 55.00% Test: 58.82%
Run: 02, Epoch: 195, Loss: 0.7302, Train: 66.67%, Valid: 50.00% Test: 64.71%
Run: 02, Epoch: 196, Loss: 0.7396, Train: 70.00%, Valid: 51.25% Test: 64.71%
Run: 02, Epoch: 197, Loss: 0.6911, Train: 70.83%, Valid: 55.00% Test: 62.75%
Run: 02, Epoch: 198, Loss: 0.6930, Train: 71.67%, Valid: 55.00% Test: 62.75%
Run: 02, Epoch: 199, Loss: 0.7282, Train: 70.83%, Valid: 55.00% Test: 62.75%

Run: 03, Epoch: 113, Loss: 0.6141, Train: 76.67%, Valid: 52.50% Test: 60.78%
Run: 03, Epoch: 114, Loss: 0.6647, Train: 78.33%, Valid: 52.50% Test: 60.78%
Run: 03, Epoch: 115, Loss: 0.6632, Train: 75.83%, Valid: 51.25% Test: 52.94%
Run: 03, Epoch: 116, Loss: 0.6749, Train: 76.67%, Valid: 52.50% Test: 52.94%
Run: 03, Epoch: 117, Loss: 0.6962, Train: 69.17%, Valid: 48.75% Test: 52.94%
Run: 03, Epoch: 118, Loss: 0.6277, Train: 66.67%, Valid: 45.00% Test: 50.98%
Run: 03, Epoch: 119, Loss: 0.6527, Train: 59.17%, Valid: 41.25% Test: 45.10%
Run: 03, Epoch: 120, Loss: 0.6522, Train: 52.50%, Valid: 35.00% Test: 37.25%
Run: 03, Epoch: 121, Loss: 0.7386, Train: 54.17%, Valid: 35.00% Test: 39.22%
Run: 03, Epoch: 122, Loss: 0.6717, Train: 68.33%, Valid: 41.25% Test: 47.06%
Run: 03, Epoch: 123, Loss: 0.6944, Train: 72.50%, Valid: 48.75% Test: 52.94%
Run: 03, Epoch: 124, Loss: 0.6346, Train: 75.00%, Valid: 50.00% Test: 56.86%
Run: 03, Epoch: 125, Loss: 0.6940, Train: 76.67%, Valid: 50.00% Test: 56.86%

...

Run: 08, Epoch: 11, Loss: 1.2502, Train: 44.17%, Valid: 47.50% Test: 50.98%
Run: 08, Epoch: 12, Loss: 1.1903, Train: 45.00%, Valid: 47.50% Test: 50.98%
Run: 08, Epoch: 13, Loss: 1.1765, Train: 44.17%, Valid: 47.50% Test: 50.98%
Run: 08, Epoch: 14, Loss: 1.1672, Train: 45.00%, Valid: 47.50% Test: 50.98%
Run: 08, Epoch: 15, Loss: 1.1804, Train: 44.17%, Valid: 48.75% Test: 50.98%
Run: 08, Epoch: 16, Loss: 1.1685, Train: 44.17%, Valid: 48.75% Test: 50.98%
Run: 08, Epoch: 17, Loss: 1.1586, Train: 44.17%, Valid: 48.75% Test: 50.98%
Run: 08, Epoch: 18, Loss: 1.1533, Train: 44.17%, Valid: 48.75% Test: 50.98%
Run: 08, Epoch: 19, Loss: 1.1834, Train: 44.17%, Valid: 48.75% Test: 50.98%
Run: 08, Epoch: 20, Loss: 1.1217, Train: 44.17%, Valid: 48.75% Test: 52.94%
Run: 08, Epoch: 21, Loss: 1.1142, Train: 43.33%, Valid: 50.00% Test: 56.86%
Run: 08, Epoch: 22, Loss: 1.1068, Train: 44.17%, Valid: 51.25% Test: 56.86%
Run: 08, Epoch: 23, Loss: 1.1349, Train: 43.33%, Valid: 51.25% Test: 58.82%
Run: 08, Epo

Run: 08, Epoch: 138, Loss: 0.9330, Train: 65.00%, Valid: 55.00% Test: 62.75%
Run: 08, Epoch: 139, Loss: 0.9349, Train: 64.17%, Valid: 53.75% Test: 60.78%
Run: 08, Epoch: 140, Loss: 0.8732, Train: 66.67%, Valid: 56.25% Test: 62.75%
Run: 08, Epoch: 141, Loss: 0.8687, Train: 62.50%, Valid: 53.75% Test: 58.82%
Run: 08, Epoch: 142, Loss: 0.9788, Train: 56.67%, Valid: 51.25% Test: 56.86%
Run: 08, Epoch: 143, Loss: 0.9104, Train: 56.67%, Valid: 55.00% Test: 58.82%
Run: 08, Epoch: 144, Loss: 0.9174, Train: 58.33%, Valid: 52.50% Test: 54.90%
Run: 08, Epoch: 145, Loss: 0.9343, Train: 53.33%, Valid: 61.25% Test: 60.78%
Run: 08, Epoch: 146, Loss: 0.9230, Train: 55.83%, Valid: 56.25% Test: 64.71%
Run: 08, Epoch: 147, Loss: 0.8885, Train: 53.33%, Valid: 53.75% Test: 62.75%
Run: 08, Epoch: 148, Loss: 0.9085, Train: 54.17%, Valid: 52.50% Test: 60.78%
Run: 08, Epoch: 149, Loss: 0.8944, Train: 55.00%, Valid: 53.75% Test: 56.86%
Run: 08, Epoch: 150, Loss: 0.8910, Train: 57.50%, Valid: 52.50% Test: 58.82%

Run: 09, Epoch: 63, Loss: 0.8491, Train: 62.50%, Valid: 48.75% Test: 45.10%
Run: 09, Epoch: 64, Loss: 0.8766, Train: 60.00%, Valid: 48.75% Test: 45.10%
Run: 09, Epoch: 65, Loss: 0.8356, Train: 64.17%, Valid: 53.75% Test: 43.14%
Run: 09, Epoch: 66, Loss: 0.8423, Train: 61.67%, Valid: 52.50% Test: 45.10%
Run: 09, Epoch: 67, Loss: 0.8708, Train: 62.50%, Valid: 51.25% Test: 49.02%
Run: 09, Epoch: 68, Loss: 0.8849, Train: 64.17%, Valid: 47.50% Test: 49.02%
Run: 09, Epoch: 69, Loss: 0.8027, Train: 61.67%, Valid: 53.75% Test: 49.02%
Run: 09, Epoch: 70, Loss: 0.8827, Train: 60.00%, Valid: 56.25% Test: 49.02%
Run: 09, Epoch: 71, Loss: 0.8608, Train: 57.50%, Valid: 57.50% Test: 52.94%
Run: 09, Epoch: 72, Loss: 0.9239, Train: 61.67%, Valid: 53.75% Test: 49.02%
Run: 09, Epoch: 73, Loss: 0.9031, Train: 64.17%, Valid: 50.00% Test: 52.94%
Run: 09, Epoch: 74, Loss: 0.8039, Train: 65.83%, Valid: 48.75% Test: 47.06%
Run: 09, Epoch: 75, Loss: 0.8411, Train: 62.50%, Valid: 51.25% Test: 45.10%
Run: 09, Epo

Run: 09, Epoch: 189, Loss: 0.6407, Train: 76.67%, Valid: 50.00% Test: 52.94%
Run: 09, Epoch: 190, Loss: 0.7015, Train: 78.33%, Valid: 50.00% Test: 49.02%
Run: 09, Epoch: 191, Loss: 0.7224, Train: 78.33%, Valid: 50.00% Test: 41.18%
Run: 09, Epoch: 192, Loss: 0.7733, Train: 78.33%, Valid: 50.00% Test: 41.18%
Run: 09, Epoch: 193, Loss: 0.6987, Train: 77.50%, Valid: 51.25% Test: 43.14%
Run: 09, Epoch: 194, Loss: 0.6898, Train: 80.00%, Valid: 48.75% Test: 47.06%
Run: 09, Epoch: 195, Loss: 0.7199, Train: 69.17%, Valid: 41.25% Test: 41.18%
Run: 09, Epoch: 196, Loss: 0.7362, Train: 75.83%, Valid: 46.25% Test: 50.98%
Run: 09, Epoch: 197, Loss: 0.7178, Train: 72.50%, Valid: 45.00% Test: 45.10%
Run: 09, Epoch: 198, Loss: 0.7628, Train: 70.00%, Valid: 46.25% Test: 45.10%
Run: 09, Epoch: 199, Loss: 0.7497, Train: 66.67%, Valid: 45.00% Test: 41.18%
Run: 09, Epoch: 200, Loss: 0.6621, Train: 65.83%, Valid: 41.25% Test: 43.14%
Run 09:
Highest Train: 80.00
Highest Valid: 57.50
  Final Train: 57.50
   Fi

Run: 10, Epoch: 114, Loss: 0.6608, Train: 69.17%, Valid: 63.75% Test: 64.71%
Run: 10, Epoch: 115, Loss: 0.8099, Train: 70.00%, Valid: 58.75% Test: 64.71%
Run: 10, Epoch: 116, Loss: 0.7028, Train: 67.50%, Valid: 56.25% Test: 58.82%
Run: 10, Epoch: 117, Loss: 0.7326, Train: 66.67%, Valid: 56.25% Test: 56.86%
Run: 10, Epoch: 118, Loss: 0.7035, Train: 66.67%, Valid: 56.25% Test: 56.86%
Run: 10, Epoch: 119, Loss: 0.8508, Train: 64.17%, Valid: 56.25% Test: 58.82%
Run: 10, Epoch: 120, Loss: 0.6983, Train: 65.83%, Valid: 57.50% Test: 60.78%
Run: 10, Epoch: 121, Loss: 0.6753, Train: 65.83%, Valid: 63.75% Test: 62.75%
Run: 10, Epoch: 122, Loss: 0.7377, Train: 70.00%, Valid: 66.25% Test: 62.75%
Run: 10, Epoch: 123, Loss: 0.7589, Train: 70.83%, Valid: 65.00% Test: 62.75%
Run: 10, Epoch: 124, Loss: 0.7297, Train: 71.67%, Valid: 62.50% Test: 62.75%
Run: 10, Epoch: 125, Loss: 0.7033, Train: 70.00%, Valid: 63.75% Test: 62.75%
Run: 10, Epoch: 126, Loss: 0.7432, Train: 69.17%, Valid: 60.00% Test: 62.75%

# Topo-GCN

In [13]:
dataset = WebKB(root='/tmp/Wisconsin', name='Wisconsin',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515])


In [16]:
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[251, 1703], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10], adj_t=[251, 251, nnz=515], topo=[251, 24])


In [21]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.0,'num_layers_mlp': 3}
    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    model = GCN(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        idx_train=[data.train_mask[i][run] for i in range(len(data.y))]
        train_idx = np.where(idx_train)[0]
        idx_val=[data.val_mask[i][run] for i in range(len(data.y))]
        valid_idx = np.where(idx_val)[0]
        idx_test=[data.test_mask[i][run] for i in range(len(data.y))]
        test_idx = np.where(idx_test)[0]
        
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                #print(f'Run: {run + 1:02d}, 'f'Epoch: {epoch:02d}, 'f'Loss: {loss:.4f}, 'f'Train: {100 * train_acc:.2f}%, '
                 #     f'Valid: {100 * valid_acc:.2f}% '
                  #    f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run 01:
Highest Train: 100.00
Highest Valid: 63.75
  Final Train: 72.50
   Final Test: 66.67
Run 02:
Highest Train: 99.17
Highest Valid: 60.00
  Final Train: 97.50
   Final Test: 52.94
Run 03:
Highest Train: 99.17
Highest Valid: 60.00
  Final Train: 72.50
   Final Test: 60.78
Run 04:
Highest Train: 99.17
Highest Valid: 53.75
  Final Train: 86.67
   Final Test: 52.94
Run 05:
Highest Train: 99.17
Highest Valid: 52.50
  Final Train: 49.17
   Final Test: 39.22
Run 06:
Highest Train: 99.17
Highest Valid: 52.50
  Final Train: 82.50
   Final Test: 60.78
Run 07:
Highest Train: 99.17
Highest Valid: 58.75
  Final Train: 73.33
   Final Test: 58.82
Run 08:
Highest Train: 100.00
Highest Valid: 57.50
  Final Train: 61.67
   Final Test: 60.78
Run 09:
Highest Train: 100.00
Highest Valid: 52.50
  Final Train: 91.67
   Final Test: 52.94
Run 10:
Highest Train: 100.00
Highest Valid: 56.25
  Final Train: 68.33
   Final Test: 54.90
All runs:
Highest Train: 99.50 ± 0.43
Highest Valid: 56.75 ± 3.92
  Final Tr